In [ ]:
# Update roboflow data set for darknet
# ROBOFLOW_ZIP_FILE="https://app.roboflow.com/ds/7viaILdWFq?key=BiHhOAv3mw" # Driver license
ROBOFLOW_ZIP_FILE="https://app.roboflow.com/ds/9itmzNBDlH?key=CKAHQ5dLuv" # PR

In [ ]:
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%cd /content/
%rm -rf dartnet-custom-object-detection
%rm -rf /content/drive/drive/MyDrive/backup
%rm -rf /content/TRAIN_DATA

In [ ]:
!curl -L {ROBOFLOW_ZIP_FILE} > roboflow.zip; unzip roboflow.zip -d TRAIN_DATA; rm roboflow.zip

In [ ]:
!git clone https://github.com/leowilbur/dartnet-custom-object-detection.git

In [ ]:
%cd /content/dartnet-custom-object-detection 
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
%cd /content/dartnet-custom-object-detection 
# We already build darknet for CodeLab
# !make
!chmod +x darknet 

In [ ]:
#Set up training file directories for custom dataset
%rm data_train -rf
%mkdir data_train
%rm backup -rf
%mkdir backup
%cd /content/dartnet-custom-object-detection/
%cp /content/TRAIN_DATA/train/_darknet.labels data_train/obj.names
%mkdir data_train/images
#copy image and labels
%cp /content/TRAIN_DATA/train/*.jpg data_train/images/
%cp /content/TRAIN_DATA/valid/*.jpg data_train/images/

%mkdir data_train/labels
%cp /content/TRAIN_DATA/train/*.txt data_train/labels/
%cp /content/TRAIN_DATA/valid/*.txt data_train/labels/

classes = sum(1 for line in open('/content/TRAIN_DATA/train/_darknet.labels'))

with open('data_train/obj.data', 'w') as out:
  out.write('classes = %s\n' % classes )
  out.write('train = data_train/train.txt\n')
  out.write('valid = data_train/valid.txt\n')
  out.write('names = data_train/obj.names\n')
  # out.write('backup = /content/drive/MyDrive/backup')
  out.write('backup = backup')

#write train file (just the image list)
import os

with open('data_train/train.txt', 'w') as out:
  for img in [f for f in os.listdir('/content/TRAIN_DATA/train') if f.endswith('jpg')]:
    out.write('data_train/images/' + img + '\n')

#write the valid file (just the image list)
import os

with open('data_train/valid.txt', 'w') as out:
  for img in [f for f in os.listdir( '/content/TRAIN_DATA/valid') if f.endswith('jpg')]:
    out.write('data/images/' + img + '\n')

In [ ]:
%cd /content/dartnet-custom-object-detection 
!./darknet detector train data_train/obj.data cfg/yolov3-tiny.cfg darknet53.conv.74